#### **WHAT?**
> Program python untuk membuat arsitektur data lakehouse sederhana diatas data lake secara lokal dengan ducklake.

---

#### **GOALS**
1. Berhasil membuat file _ducklake_ secara lokal.
2. Ducklake berrhasil membuat tabel dari data di MinIO dan mulai menyimpannya di `./managedStoreData/`.
3. Ketika ada update ducklake membuat file parquet baru dengan versi yang terbaru.
4. Bisa mengakses data dengan versi yang diinginkan langsung via query SQL ke catalognya.

---

#### **FLOW PROGRAM:**

Import library yang dibutuhkan

In [1]:
import duckdb

1. Install _**Ducklake**_ dan _**httpfs**_ via DuckDB:
> **DuckLake**: Extension DuckDB yang digunakan untuk mengelola metadata (informasi tentang data) dan menyediakan fitur yang berguna seperti time-travel seperti di `git` yang menyimpan log commit beserta (nama pengubah, tanggal, apa yang diubah, di file mana, dll)

> **httpfs**: Ekstensi httpfs adalah ekstensi yang memungkinkan pembacaan/penulisan berkas jarak jauh (melalui http endpoint). Untuk HTTP(S) biasa, hanya read file yang didukung. Untuk penyimpanan objek menggunakan API S3 (seperti MinIO Object Storage), ekstensi httpfs mendukung pembacaan/penulisan/globbing berkas.

In [2]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")
con.sql("INSTALL ducklake; LOAD ducklake;")

2. Melakukan konfigurasi untuk menghubungkan ke **MinIO Object Storage** dengan:
    - **Set Endpoint MinIO** (API endpoint untuk terhubung ke server MinIO)
    - **Set Access Key MinIO** (username MinIO) sebagai Access Key s3 (karena MinIO bisa pakai API s3)
    - **Set Secret Ket MinIO** (password MinIO) sebagai Secret Key s3 (karena MinIO bisa pakai API s3)
    - **Set ssl s3** sebagai `false` karena endpoint kita http jadi tanpa ssl
    - **Set url style** sebagai path untuk memaksa sistem menggunakan format URL berbasis path (/bucket/object) saat mengakses MinIO. Ini dilakukan agar koneksi lebih andal dan bekerja langsung di lingkungan non-AWS, karena menghindari format URL virtual hosted yang sering gagal tanpa konfigurasi DNS dan SSL yang rumit.

In [ ]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

minioEndPoint = minioAuth['endpoint'].replace('http://', '')
con.sql(f"SET s3_endpoint = '{minioEndPoint}';") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
# 📌 MATIKAN SSL dan gunakan STYLE PATH
con.sql("SET s3_use_ssl = false;") # Wajib untuk http (tanpa ssl)
con.sql("SET s3_url_style = 'path';") # Wajib untuk MinIO

3. Menyiapkan path file untuk:
    - Tempat file database catalog ducklake untuk mengelola metadata di `./catalog/catalog.ducklake`
    - Tempat kumpulan file yang dikelola oleh duck lake (untuk berbagai version juga) `./managedStoreData/`
    - Lokasi sumber object/file dari data lake MinIO untuk mulai di kelola di lingkungan ducklake

In [4]:
# Path Metadata Catalog
catalogMetadata = './catalog/catalog.ducklake'

# Path Folder dimana file dikelola oleh catalog ducklake
managedStoredData = './managedStoreData/'

# Daftar file yang akan dikelola oleh catalog ducklake
minioObjectName = {
    'jakarta': 'staging/parquet/duckdb/JawaBarat/jakarta.parquet',
    'bekasi': 'staging/parquet/duckdb/JawaBarat/bekasi.parquet',
    'depok': 'staging/parquet/duckdb/JawaBarat/depok.parquet',
    'balikpapan': 'staging/parquet/duckdb/kalimantanTimur/balikpapan.parquet',
    'makassar': 'staging/parquet/duckdb/sulawesiSelatan/makassar.parquet',
    'palembang': 'staging/parquet/duckdb/sumateraSelatan/palembang.parquet'
}

4. Menyiapkan Endpoint URL s3 yang lengkap dengan format (contoh Jakarta)
`s3://data-dapodik-kemendikdasmen/staging/parquet/duckdb/JawaBarat/jakarta.parquet`

In [23]:
fullEndPointURL = {
    'jakarta': f"s3://{minioAuth['bucket']}/{minioObjectName['jakarta']}",
    'bekasi': f"s3://{minioAuth['bucket']}/{minioObjectName['bekasi']}",
    'depok': f"s3://{minioAuth['bucket']}/{minioObjectName['depok']}",
    'balikpapan': f"s3://{minioAuth['bucket']}/{minioObjectName['balikpapan']}",
    'makassar': f"s3://{minioAuth['bucket']}/{minioObjectName['makassar']}",
    'palembang': f"s3://{minioAuth['bucket']}/{minioObjectName['palembang']}"
}

5. Membuat file _ducklake_ dengan:
    - ATTACH di directory `./catalog/catalog.ducklake` (membuat database duckdb dengan skema ducklake)
    - Penyimpanan data update nya pada path `./managedStoreData/`
    - Lalu kita gunakan file database yang kita buat dengan `USE` statement

In [5]:
con.sql(f"ATTACH 'ducklake:{catalogMetadata}' AS LocalLake (DATA_PATH '{managedStoredData}');")
con.sql("USE LocalLake;")

6. Kita Buat schema yang bernama `staging` terlebih dahulu jika belum ada di dalam file database tersebut

In [ ]:
con.sql("CREATE SCHEMA IF NOT EXISTS staging;")

7. Membuat Variabel untuk menampung query SQL:
Buat atau update table dari skema staging sebagai Pilih Semua dari data dile parquet di MinIO

In [25]:
loadJakarta = f"""
        CREATE OR REPLACE TABLE staging.dataJakarta AS 
        SELECT * FROM read_parquet('{fullEndPointURL['jakarta']}');
    """
loadBekasi = f"""
        CREATE OR REPLACE TABLE staging.dataBekasi AS 
        SELECT * FROM read_parquet('{fullEndPointURL['bekasi']}');
    """
loadDepok = f"""
        CREATE OR REPLACE TABLE staging.dataDepok AS 
        SELECT * FROM read_parquet('{fullEndPointURL['depok']}');
    """
loadBalikpapan = f"""
        CREATE OR REPLACE TABLE staging.dataBalikpapan AS 
        SELECT * FROM read_parquet('{fullEndPointURL['balikpapan']}');
    """
loadMakassar = f"""
        CREATE OR REPLACE TABLE staging.dataMakassar AS 
        SELECT * FROM read_parquet('{fullEndPointURL['makassar']}');
    """
loadPalembang = f"""
        CREATE OR REPLACE TABLE staging.dataPalembang AS 
        SELECT * FROM read_parquet('{fullEndPointURL['palembang']}');
    """

8. Jalankan query SQL yang ditampung di variabel dan tangkap penyebab error pada output jika terjadi error

In [26]:
try:
    con.sql(loadJakarta)
    con.sql(loadBekasi)
    con.sql(loadDepok)
    con.sql(loadBalikpapan)
    con.sql(loadMakassar)
    con.sql(loadPalembang)
    print(f"✅ Semua tabel staging terdaftar. (Source: MinIO, Managed: Lokal)")

except Exception as e:
    print(f"❌ Gagal memuat tabel | Error: {e}")

✅ Semua tabel staging terdaftar. (Source: MinIO, Managed: Lokal)


In [15]:
print("\n--- Time Travel ke Data Depok (Versi 0) ---")
time_travel_query = f"""
    SELECT 
        *
    FROM 
        ducklake_snapshots('LocalLake');
"""
con.sql(time_travel_query)


--- Time Travel ke Data Depok (Versi 0) ---


┌─────────────┬───────────────────────────────┬────────────────┬─────────────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                               changes                               │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                       map(varchar, varchar[])                       │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼─────────────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2025-12-04 21:42:38.715397+07 │              0 │ {schemas_created=[main]}                                            │ NULL    │ NULL           │ NULL              │
│           1 │ 2025-12-04 21:42:38.879408+07 │              1 │ {schemas_created=[staging

In [16]:
print("\n--- Time Travel ke Data Depok (Versi 0) ---")
time_travel_query = f"""
    SELECT 
        *
    FROM 
        staging.dataJakarta
    AT (VERSION => 2);
"""
con.sql(time_travel_query)


--- Time Travel ke Data Depok (Versi 0) ---


┌────────────────────────┬────────────────┬─────────────────────────┬───────────────────────┬──────────┬─────────┬───────────────────┬────────────────────┬───────────────────────────┬──────────────────────┬─────────────────────┬─────────────────────────────┬───────────────────────────┬───────────┬────────────────────────────────────────────┬──────────────────────────┬────────────┬──────────────────────┬─────────────────┬────────────────┬──────────────┬────────────────────┬──────────────────────────┬────────────────────────┬────────────┬───────────────────┬─────────┬────────────────────────────────────────┬─────────┬─────────────────────────┬────────────────────────┬────────────────────────┬─────────────────────────┬──────────────────────┬──────────┬─────────┬──────────┬────────┬────────┬────────────┬──────────┬──────────┬──────────────┬───────┬───────┬───────────┬───────┬───────┬──────────┬────────────────────┬───────────────────┬─────────────────────┬────────────────────┬─────────────

update

In [17]:
con.sql("""
        UPDATE
            staging.dataJakarta
        SET 
            Provinsi = 'Jawa Tengah'
        WHERE
            Nama_Sekolah = 'SDN DUREN SAWIT 01' AND NPSN = '20108557';
""")

In [ ]:
time_travel_query = f"""
    SELECT 
        *
    FROM 
        staging.dataJakarta
    AT (VERSION => 8)
    WHERE NPSN = '20108557';
"""
con.sql(time_travel_query)


--- Time Travel ke Data Depok (Versi 0) ---


┌────────────────────┬─────────────┬─────────────────────────┬───────────────────────┬──────────┬─────────┬───────────────────┬────────────────────┬───────────────────────────┬──────────────────────┬─────────────────────┬─────────────────────────────┬───────────────────────────┬───────────┬──────────────────────────────┬───────────────────────┬────────────┬──────────────────────┬─────────────────┬────────────────┬──────────────┬────────────────────┬──────────────────────────┬───────────────────────┬────────────┬───────────────────┬─────────┬───────────────────┬─────────┬─────────┬───────────────────────┬───────────────────────┬─────────────────────────┬──────────────────────┬──────────┬─────────┬──────────┬────────┬────────┬────────────┬──────────┬──────────┬──────────────┬───────┬───────┬───────────┬───────┬───────┬──────────┬────────────────────┬───────────────────┬─────────────────────┬────────────────────┬──────────────────┬─────────────────┬─────────────────────┬──────────────────

In [22]:
con.close()